In [178]:
import pandas as pd
import os
from ensemble_boxes import *
import numpy as np

In [230]:
model_path = '../models'
ensemble_list = os.listdir(model_path)
models = [pd.read_csv(os.path.join(model_path, _)) for _ in ensemble_list]
ensemble_list

['494.csv', '538.csv', '503.csv']

In [231]:
boxes_list = [[[] for i in range(len(models))] for _ in range(4871)]
scores_list = [[[] for i in range(len(models))] for _ in range(4871)]
labels_list = [[[] for i in range(len(models))] for _ in range(4871)]

In [232]:
for model_num, model in enumerate(models):
    for img_num, img in enumerate(model['PredictionString']):
        try:
            img = img.split()
        except:
            continue
        for pos in range(0, len(img), 6):
            obj = list(map(float, img[pos:pos+6]))
            label, score, box = int(obj[0]), float(obj[1]), np.array(obj[2:]) / 1024
            boxes_list[img_num][model_num].append(box)
            scores_list[img_num][model_num].append(score)
            labels_list[img_num][model_num].append(label)


In [233]:
iou_thr = 0.4
skip_box_thr = 0.0001
sigma = 0.1

In [234]:
for i in range(len(boxes_list)):
    boxes, scores, labels = weighted_boxes_fusion(
                                                boxes_list[i], scores_list[i], labels_list[i], 
                                                iou_thr=iou_thr, skip_box_thr=skip_box_thr
                                                )
    boxes_list[i], scores_list[i], labels_list[i] = boxes * 1024, scores, labels   

In [235]:
predictionstring = []
file_names = []
for img_predict in range(len(boxes_list)):
    file_names.append(models[0]['image_id'][img_predict])
    prediction_string = ''
    for label, score, box in zip(labels_list[img_predict], scores_list[img_predict], boxes_list[img_predict].tolist()):
        prediction = [str(label)] + [str(score)] + list(map(str, box))
        prediction_string += ' '.join(prediction) + ' '
    predictionstring.append(prediction_string)

In [236]:
submission = pd.DataFrame()
submission['PredictionString'] = predictionstring
submission['image_id'] = file_names
submission.to_csv(os.path.join('../ensemble/ensemble.csv'), index=None)
submission.head()

,PredictionString,image_id
0,7.0 0.999428391456604 601.418212890625 510.033...,test/0000.jpg
1,5.0 0.9586414098739624 139.74026489257812 1.43...,test/0001.jpg
2,1.0 0.9560907483100891 424.93572998046875 266....,test/0002.jpg
3,9.0 0.8247147798538208 109.2036361694336 202.4...,test/0003.jpg
4,0.0 0.42736950516700745 416.55792236328125 373...,test/0004.jpg


In [240]:
print(len(submission['PredictionString'][0].split()) // 6)
for i in range(3):
    print(len(models[i]['PredictionString'][0].split()) // 6)

101
61
75
78
